In [ ]:
!nvidia-smi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

In [ ]:
config_path = '/root/data/models/keypoints_detection/2019_03_27_22_28_27/config.json' 
checkpoint_path = '/root/data/models/keypoints_detection/2019_03_27_22_28_27/weights/fish_test-49'

In [ ]:
config = json.load(open(config_path))
config

In [ ]:
class FLAGS(object):
    """ """
    """
    General settings
    """
    input_size = tuple(config["input_size"])
    stages = 3
    joints = 8
    model_path = checkpoint_path
    cmap_radius = 21
    batch_size = 1
    keypoints_order = ["TAIL_NOTCH",
                    "ADIPOSE_FIN",
                    "UPPER_LIP",
                    "ANAL_FIN",
                    "PELVIC_FIN",
                    "EYE",
                    "PECTORAL_FIN",
                    "DORSAL_FIN"]
    normalize = True
    heatmap_size = config["heatmap_size"]
    joint_gaussian_variance = 1.0
    augmentation = None

## LOAD MODEL

In [ ]:
import tensorflow as tf
import importlib
from models.nets import fish_test
from utils import cpm_utils
import numpy as np

In [ ]:
from tensorflow.python import pywrap_tensorflow
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
# print_tensors_in_checkpoint_file(checkpoint_path, "", False,
#                                      all_tensor_names=True)

In [ ]:
# reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
# var_to_shape_map = reader.get_variable_to_shape_map()
# for key in sorted(var_to_shape_map):
#     print("tensor_name: ", key, reader.get_tensor(key).shape)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
tf_device = '/gpu:0'
with tf.device(tf_device):
    model = fish_test.CPM_Model(FLAGS.input_size, 2, FLAGS.stages, FLAGS.joints)

Restore weights

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()

sess.run(tf.global_variables_initializer())
saver.restore(sess, FLAGS.model_path)

In [ ]:
test_center_map = cpm_utils.gaussian_img(FLAGS.input_size[0],
                                         FLAGS.input_size[0], 
                                         FLAGS.input_size[0] / 2,
                                         FLAGS.input_size[0] / 2,
                                         FLAGS.cmap_radius)
test_center_map = np.reshape(test_center_map, [1, FLAGS.input_size[0], FLAGS.input_size[0], 1])

Data generator

In [ ]:
import glob
import json
import random
import matplotlib.pyplot as plt
import cv2
from utils.utils import DataGenerator, load_image_keypoints

In [ ]:
random.seed(258)

In [ ]:
annotations = glob.glob("/root/data/gtsf_phase_I/*/*_cogito_annotations.json")
print("Total number of gtsf sessions: {}".format(len(annotations)))

In [ ]:
cutoff = int(len(annotations)*0.8)
random.shuffle(annotations)
train_files = annotations[:cutoff]
val_files = annotations[cutoff:]
print("Number of train files: {}".format(len(train_files)))
print("Number of validation files: {}".format(len(val_files)))

In [ ]:
val_annotations = []
for jpath in val_files:
    val_annotations += json.load(open(jpath))
print("Number of validation data: {}".format(len(val_annotations)))
val_annotations = [ann for ann in val_annotations if ann["species"] == "salmon"]
print("Number of validation data: {}".format(len(val_annotations)))

In [ ]:
random_ann = np.random.choice(val_annotations)
img_input, kp_resized = load_image_keypoints(random_ann, FLAGS)
img_input  = img_input / 255.0 - 0.5
img_input = img_input[np.newaxis, ...]
image, keypoints = load_image_keypoints(random_ann, FLAGS, reshape=False)
height, width, _ = image.shape

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(image)
plt.scatter(keypoints[:, 0], keypoints[:, 1], c="r")
plt.show()

Froward pass

In [ ]:
with tf.device(tf_device):   
    predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                              model.stage_heatmap,
                                              ],
                                             feed_dict={'input_placeholder:0': img_input,
                                                        'cmap_placeholder:0': test_center_map})

In [ ]:
final_stage_heatmap = predict_heatmap.squeeze()

Display

In [ ]:
for i in range(8):
    plt.imshow(final_stage_heatmap[..., i])
    plt.show()

In [ ]:

print(final_stage_heatmap.shape)
f, ax = plt.subplots(5, 2, figsize=(20, 30))
c = 0
for i in range(5):
    for j in range(2):
        if c == 9:
            continue
        hm = cv2.resize(final_stage_heatmap[..., c], FLAGS.input_size)
        hm_max = np.where(hm == hm.max())
        ax[i,j].imshow(img_input.squeeze())
        ax[i,j].imshow(hm, alpha=0.5)
        if i < 4:
            ax[i,j].scatter(hm_max[1], hm_max[0], c="r")
        ax[i,j].axis("off")
        c+=1
plt.show()

In [ ]:
# final_stage_heatmap = predict_heatmap.squeeze()
# print(final_stage_heatmap.shape)
# f, ax = plt.subplots(5, 2, figsize=(20, 30))
# c = 0
# for i in range(5):
#     for j in range(2):
#         if c == 9:
#             continue
#         hm = cv2.resize(final_stage_heatmap[..., c], (4096, 3000))
#         hm_max = np.where(hm == hm.max())
#         ax[i,j].imshow(image)
#         ax[i,j].imshow(hm, alpha=0.5)
#         if i < 4:
#             ax[i,j].scatter(hm_max[1], hm_max[0], c="r")
#         ax[i,j].axis("off")
#         c+=1
# plt.show()

Calculate accuracy on full size image

In [ ]:
# manhattan distance
for c in range(8):
    hm = cv2.resize(final_stage_heatmap[..., c], FLAGS.input_size)
    hm_max = np.where(hm == hm.max())
    pred_kp = np.array([hm_max[1][0], hm_max[0][0]])
    gt_kp = np.array(kp_resized[c, :], dtype=np.uint32)
    man_dist = np.sum(np.abs(pred_kp - gt_kp))
    print("Manhattan distance between pred and gt {} for {}".format(man_dist, FLAGS.keypoints_order[c]))

In [ ]:
# manhattan distance
for c in range(8):
    hm = cv2.resize(final_stage_heatmap[..., c], (width, height))
    hm_max = np.where(hm == hm.max())
    pred_kp = np.array([hm_max[1][0], hm_max[0][0]])
    gt_kp = np.array(keypoints[c, :], dtype=np.uint32)
    man_dist = np.sum(np.abs(pred_kp - gt_kp))
    print("Manhattan distance between pred and gt {} for {}".format(man_dist, FLAGS.keypoints_order[c]))

In [ ]:
# final_stage_heatmap = predict_heatmap.squeeze()
# print(final_stage_heatmap.shape)
# f, ax = plt.subplots(1, figsize=(20, 10))
# ax.imshow(image)
# for c in range(8):
#     hm = cv2.resize(final_stage_heatmap[..., c], (width, height))
#     hm_max = np.where(hm == hm.max())
#     ax.scatter(hm_max[1], hm_max[0], c="r")
# ax.axis("off")
# plt.show()